In [26]:
import getpass
import os

from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains import create_retrieval_chain

In [3]:
dir = r'C:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\outdir\text.txt'
loader=TextLoader(dir, encoding= 'utf-8')
text=loader.load()
text

[Document(metadata={'source': 'C:\\Users\\Administrator\\Desktop\\Backend-LLM\\audio_text_llm\\SenseVoiceSmall\\outdir\\text.txt'}, page_content='应对抑郁症的复杂情感和挑战可能是一个极其艰巨的过程，尤其是在你作为学生需要平衡学术责任、社交压力和个人期望的时候。理解抑郁症不是弱点或无法应对的表现，而是一种需要同情、理解和适当护理的医学状况，这一点至关重要。在你经历这些困难时，认识和验证你的感受是很重要的。学业压力、社交动态和自我施加的期望可能会让人感到难以承受。请记住，你并不是孤单面对这些经历。许多学生也经历类似的困扰，寻求支持是迈向康复的勇敢和重要的一步。\n\n抑郁症常常扭曲我们对自己和周围情况的看法，让我们感到孤立无援，仿佛我们的困境无法克服。然而，这些感受虽然非常真实，但并不定义你的价值或改善的潜力。寻求帮助是至关重要的，无论是通过专业咨询、支持小组还是信任的亲友。治疗提供了一个安全的空间，让你可以在没有评判的情况下探索你的思想和感受，提供管理和缓解抑郁症状的工具和策略。例如，认知行为疗法可以帮助你识别和挑战那些加重抑郁症的负面思维模式和行为。此外，药物治疗可能会被建议，以帮助平衡大脑中的化学物质，但这应始终与医疗提供者充分讨论。\n\n在治疗中，我们一起探讨抑郁症的根本原因，这可能包括过去的创伤、持续的压力源或其他心理因素。我们可以深入这些领域，以帮助你制定应对策略和建立韧性。同样，关注自我护理实践也很重要，例如保持健康的日常生活、参与带给你快乐的活动、以及练习正念和放松技巧。定期的身体活动、均衡饮食和充足睡眠对支持心理健康至关重要。参与爱好、与支持的朋友和家人共度时光，以及设定小而可实现的目标也可以显著改善你的整体幸福感。\n\n请记住，管理抑郁症的进展并非总是线性的。这过程中会有起伏，这很正常。重要的是你继续迈向康复的步伐，并在需要时寻求支持。了解抑郁症和心理健康也是很有益的，它可以赋予你力量，减少无助感。你的旅程是独特的，重要的是以耐心和自我同情来对待它。感觉好转的道路往往是渐进的，但通过正确的支持和策略，你可以取得有意义的进展。\n\n你的感受是有效的，寻求帮助是积极而坚强的一步。继续寻求支持，无论是向心理

In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(text)

In [8]:
# documents = [Document(page_content=chunk) for chunk in splits]
vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [17]:
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessage, HumanMessage

# Define the template with proper message formatting
messages = [
    SystemMessage(content="""
        你是一位刚刚结束了与学生咨询的心理医生。以下是咨询的详细信息：
        <Context>  
        {context}
        <Context>
        根据这次咨询的信息，请尽可能准确和富有同情心地回答以下问题，反映学生在咨询中的行为和回答。
    """),
    HumanMessage(content="{input}")
]

# Create the ChatPromptTemplate
prompt = ChatPromptTemplate(messages=messages)


In [27]:
template = """
你是一位刚刚结束了与学生咨询的心理医生。以下是咨询的详细信息：
<Context>  
{context}
<Context>
根据这次咨询的信息，请尽可能准确和富有同情心地回答以下问题，反映学生在咨询中的行为和回答。
问题：{input}
"""

# Create the PromptTemplate
prompt = ChatPromptTemplate.from_template(
    template=template
)



document_chain=create_stuff_documents_chain(llm,prompt)

retreival_chain=create_retrieval_chain(retriever,document_chain)

In [30]:
retreival_chain.invoke({"input":"对来访者的客观描述（咨询师的观察、测试结果、医院就诊结果等）"})['answer']

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'根据本次咨询的信息，来访者表现出对抑郁症的复杂情感和挑战感到困扰，并认识到自己需要帮助和支持。他/她可能在学业压力、社交压力和个人期望下感到难以承受，表现出对自己和周围情况的看法扭曲，以及对困境无法克服的感受。在咨询中，来访者可能表现出愿意探索自己的思想和感受，并愿意寻求专业帮助和支持的积极态度。可能还会讨论过去的创伤、持续的压力源或其他心理因素，并展现出愿意接受心理治疗和改善自我护理实践的意愿。\n\n根据咨询师的观察和分析，来访者可能会被建议进行认知行为疗法和/或药物治疗，以帮助管理和缓解抑郁症状。建议来访者定期进行身体活动、保持健康的日常生活、参与带来快乐的活动，并与支持的朋友和家人共度时光。在治疗过程中，来访者可能会面对起伏和挑战，但重要的是继续迈向康复的步伐，并寻求支持。整体上，来访者表现出对自己的情感和康复过程有一定的认识和理解，愿意接受帮助并迈向更好的心理健康状态。'

In [18]:
# Define the prompt


# Define the format_docs function
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Update the RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [19]:
rag_chain.invoke("对来访者的客观描述（咨询师的观察、测试结果、医院就诊结果等）")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'学生可能会感到沮丧和失望，因为他们对自己的未来感到迷茫，不知道自己要做什么或者将来会怎样。在咨询中，学生可能会表达出对未来的焦虑和不安，可能会感到压力很大，无法找到方向。他们可能会提到对现状感到沮丧，无法找到实现自己目标的方法。在咨询中，学生可能会试图寻求帮助和支持，希望找到一种解决困境的方法。\n\n针对这种情况，作为心理医生，我会倾听学生的倾诉，帮助他们理清自己的思路和情绪，引导他们找到解决问题的方法。我会鼓励学生积极面对困难，提出一些实际的建议和行动计划，帮助他们重新找回方向和信心。同时，我也会对学生表现出的情绪和困惑表示同情和理解，让他们感受到支持和安慰。'